In [ ]:
from toraniko.ticker_universe import get_tickers, UNIVERSE_SP500, UNIVERSE_RUSSELL3000

# ── Universe selection ──────────────────────────────────────────
UNIVERSE = "sp500"          # "sp500" | "russell3000"

tickers = get_tickers(UNIVERSE)
print(f"Universe: {UNIVERSE}")
print(f"Tickers: {len(tickers)}")
print(f"First 20: {tickers[:20]}")